支持：  
    "resnet18_3D"   "resnet10_3D"   "resnet34_3D"   "resnet50_3D"    "resnet101_3D"  "resnet152_3D"     "resnet200_3D"             
    "DenseNet121_3D"   "DenseNet201_3D"  "DenseNet169_3D"    "DenseNet264_3D"  

In [ ]:
import os
import torch
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.medical_imaging.modelstore import get_model_from_name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
#可改参数
model='resnet50_3D'
num_classes=2
pretrained=True
in_channels=1
batch_size=4
learning_rate=1e-4
epochs=10

model=get_model_from_name['DenseNet121_3D'](num_classes = 2, pretrained = True,in_channels=1).to(device)


In [ ]:
from pixelmed_calc.medical_imaging.DLtools.process import load_data_from_csv,get_file_and_labels,get_transforms,train_and_validate
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import torch
import torch.nn as nn
from monai.data import ImageDataset
import os
import pandas as pd
from torch.utils.data import DataLoader

# CSV文件路径
csv_path = r"D:\pixelmedAI\function_pm\pm_data\liangxing\group.csv"#'E:\127lungnode\lungnode\group.csv'
train_base_path = r"D:\pixelmedAI\function_pm\pm_data\liangxing\images"#'E:\127lungnode\lungnode\crop3d'
val_base_path = r"D:\pixelmedAI\function_pm\pm_data\liangxing\images"#'E:\127lungnode\lungnode\crop3d'
# CSV文件路径
tfs=get_transforms()
# 读取所有数据
file_names, labels, groups = load_data_from_csv(csv_path)
# 获取训练数据和标签（可以设置 base_path 为 None 或实际路径）
file_train, train_labels = get_file_and_labels(file_names, labels, groups, 'train', train_base_path)
# 获取验证数据和标签（可以设置 base_path 为 None 或实际路径）
file_val, val_labels = get_file_and_labels(file_names, labels, groups, 'val', val_base_path)

# 假设你已经定义了 ImageDataset 和 train_transforms, val_transforms
train_ds = ImageDataset(image_files=file_train, labels=train_labels, transform=tfs)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

val_ds = ImageDataset(image_files=file_val, labels=val_labels, transform=tfs)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

In [ ]:
file_val

训练模型

In [ ]:
train_and_validate(model, train_loader, val_loader, learning_rate=1e-4, epochs=50)

可视化

In [ ]:
import matplotlib.pyplot as plt  
import torch  
from pixelmed_calc.medical_imaging.DLtools.process import plot_gradcam

model = get_model_from_name['DenseNet121_3D'](num_classes=2, pretrained=False, in_channels=1).to(device)

model.load_state_dict(torch.load(r'd:\pixelmedAI\function_pm\note2-深度学习分类\results\latest_model.pth'))
for name, _ in model.named_modules(): print(name)


In [ ]:
# 示例调用:  
plot_gradcam(model, file_val[1:2], tfs, device,target_layers="features.denseblock4.denselayer15.conv2")